In [57]:
import pandas as pd
import re
from collections import defaultdict
import textwrap

In [58]:
INPUT_FILE = "../analysis-tja/plots_html/UNIT1/formulas_multivariate.csv"
WRAP_WIDTH = 150

In [59]:
df = pd.read_csv(INPUT_FILE)
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df.reset_index(drop=True, inplace=True)
df = df.drop_duplicates(subset=["area_name", "tag_name"], keep="first").reset_index(drop=True)

In [60]:
# Parse formula linear
def parse_linear_formula(formula: str):
    lhs, rhs = formula.split("=", 1)
    target = lhs.strip()
    rhs = rhs.replace("-", "+ -")
    parts = [p.strip() for p in rhs.split("+") if p.strip()]

    intercept = 0.0
    terms = []

    for p in parts:
        if re.fullmatch(r"-?\d+(\.\d+)?", p):
            intercept = float(p)
            continue
        if "*" in p:
            coef, tag = p.split("*", 1)
            try:
                float(coef.strip())
                terms.append(f"{coef.strip()}*({tag.strip()})")
            except ValueError:
                pass

    model_eq = str(intercept)
    if terms:
        model_eq += " + " + " + ".join(terms)

    model_eq = model_eq.replace("+ -", "- ")
    return target, model_eq

In [61]:
# dictionary RULES per area dan tag
RULES = defaultdict(lambda: defaultdict(list))
for _, r in df.iterrows():
    target, model_eq = parse_linear_formula(r["formula"])
    logic_expr = f"( {target} {r['simbol']} {model_eq} )"
    RULES[r["area_name"]][r["tag_name"]].append(logic_expr)

In [62]:
# Fungsi tag/block
def wrap_tag(expr):
    return f"({expr})" if not expr.startswith("(") else expr

def build_block(expr_list, op="AND"):
    if len(expr_list) == 1:
        return wrap_tag(expr_list[0])
    return f"({f' {op} '.join(expr_list)})"

def build_engineer_logic(rules_by_area, area, block_definitions, outer_op="OR"):
    blocks_expr = []
    for block in block_definitions:
        tags = block["group"]
        op = block.get("op", "AND")
        tag_exprs = []
        for tag in tags:
            if tag not in rules_by_area[area]:
                raise ValueError(f"Tag '{tag}' tidak ada di area '{area}'")
            tag_exprs.append(wrap_tag(rules_by_area[area][tag][0]))
        blocks_expr.append(build_block(tag_exprs, op))
    return build_block(blocks_expr, outer_op)

In [63]:
# ENGINEER_LOGIC_BY_AREA
ENGINEER_LOGIC_BY_AREA = [
    # ---------------- FURNACE BACK AND RIGHT ----------------
    {
        "area": "FURNACE BACK AND RIGHT",
        "expression": [
            {"group": ["SH.FIN.OUT HEADER R OUT GAS TEMP", "STEAM FLOW"], "op": "AND"},
            {"group": ["STEAM TEMP OUT PLATEN SH"]},
            {"group": ["SH 1ST SPRAY WATER FLOW"]}
        ],
        "outer_op": "OR"
    },
    # ---------------- FURNACE BACK AND LEFT ----------------
    {
        "area": "FURNACE FRONT AND LEFT",
        "expression": [
            {"group": ["SH.FIN.OUT HEADER L OUT GAS TEMP", "STEAM FLOW"], "op": "AND"},
            {"group": ["STEAM TEMP OUT PLATEN SH"]},
            {"group": ["SH 1ST SPRAY WATER FLOW"]}
        ],
        "outer_op": "OR"
    },
    # ---------------- PANEL SH L ----------------
    {
        "area": "PANEL SH L",
        "expression": [
            {"group": ["SH.FIN.OUT HEADER L OUT GAS TEMP", "SH 1ST SPRAY WATER FLOW"], "op": "AND"},
            {"group": ["SH.FIN.OUT HEADER L OUT GAS TEMP", "SH 2ND SPRAY IN STM TEMP(L)"], "op": "AND"},
            {"group": ["DELTA STEAM TEMP AT PANEL-PLATEN SH"]}
        ],
        "outer_op": "OR"
    },
    # ---------------- PANEL SH R ----------------
    {
        "area": "PANEL SH R",
        "expression": [
            {"group": ["SH.FIN.OUT HEADER R OUT GAS TEMP", "SH 1ST SPRAY WATER FLOW"], "op": "AND"},
            {"group": ["SH.FIN.OUT HEADER R OUT GAS TEMP", "SH 2ND SPRAY IN STM TEMP(R)"], "op": "AND"},
            {"group": ["DELTA STEAM TEMP AT PANEL-PLATEN SH"]}
        ],
        "outer_op": "OR"
    },
    # ---------------- REHEAT L ----------------
    {
        "area": "REHEAT L",
        "expression": [
            {"group": ["L RH OUT STM TEMP", "SH.FIN.OUT HEADER L OUT GAS TEMP"], "op": "AND"},
            {"group": ["MAIN STM TEMP", "SH.FIN.OUT HEADER L OUT GAS TEMP"], "op": "AND"},
            {"group": ["Delta Steam Temp Platen RH - Final RH"]}
        ],
        "outer_op": "OR"
    },
    # ---------------- REHEAT R ----------------
    {
        "area": "REHEAT R",
        "expression": [
            {"group": ["R RH OUT STM TEMP", "SH.FIN.OUT HEADER R OUT GAS TEMP"], "op": "AND"},
            {"group": ["MAIN STM TEMP", "SH.FIN.OUT HEADER R OUT GAS TEMP"], "op": "AND"},
            {"group": ["Delta Steam Temp Platen RH - Final RH"]}
        ],
        "outer_op": "OR"
    },
    # ---------------- LTSH L ----------------
    {
        "area": "LTSH L",
        "expression": [
            {"group": ["SH 1ST SPRAY IN STM TEMP(L)", "SH.FIN.OUT HEADER L OUT GAS TEMP"], "op": "AND"},
            {"group": ["Delta Pressure"]},
            {"group": ["Delta Temperatur"]}
        ],
        "outer_op": "OR"
    },
    # ---------------- LTSH R ----------------
    {
        "area": "LTSH R",
        "expression": [
            {"group": ["SH 1ST SPRAY IN STM TEMP(R)", "SH.FIN.OUT HEADER R OUT GAS TEMP"], "op": "AND"},
            {"group": ["Delta Pressure"]},
            {"group": ["Delta Temperatur"]}
        ],
        "outer_op": "OR"
    },
    # ---------------- ECONOMIZER L ----------------
    {
        "area": "ECONOMIZER L",
        "expression": [
            {"group": ["Delta Pressure"]},
            {"group": ["Delta Temperatur"]},
            {"group": ["LMTD"]}
        ],
        "outer_op": "OR"
    },
    # ---------------- ECONOMIZER R ----------------
    {
        "area": "ECONOMIZER R",
        "expression": [
            {"group": ["Delta Pressure"]},
            {"group": ["Delta Temperatur"]},
            {"group": ["LMTD"]}
        ],
        "outer_op": "OR"
    },
    # ---------------- APH L ----------------
    {
        "area": "APH L",
        "expression": [
            {"group": ["Delta Pressure"]},
            {"group": ["Delta Temperatur"]},
            {"group": ["LMTD"]}
        ],
        "outer_op": "OR"
    },
    # ---------------- APH R ----------------
    {
        "area": "APH R",
        "expression": [
            {"group": ["Delta Pressure"]},
            {"group": ["Delta Temperatur"]},
            {"group": ["LMTD"]}
        ],
        "outer_op": "OR"
    }
]

In [64]:
# Build semua area otomatis
FINAL_RULES = {}
for area_def in ENGINEER_LOGIC_BY_AREA:
    area = area_def["area"]
    final_expr = build_engineer_logic(RULES, area, area_def["expression"], outer_op=area_def.get("outer_op", "OR"))
    FINAL_RULES[area] = final_expr

    print("="*100)
    print(area, ":\n")
    print(textwrap.fill(final_expr, WRAP_WIDTH))
    print()

FURNACE BACK AND RIGHT :

((( EWS102/10DAS0E:TC1E047203.PNT > 34.706 + 0.096*(EWS102/10DAS0O:AI1O013305.PNT) + 1.018*(EWS102/10DAS0D:TC1D027203.PNT) ) AND (
EWS102/10DISP:STEAMFLOWB.PNT < -69.301 + 1.084*(EWS102/10DAS0O:AI1O013305.PNT) + 254.241*(EWS102/10DAS0J:PT1J010805.PNT) )) OR (
EWS102/10DAS0A:TC1A012101.PNT > -86.282 - 0.055*(EWS102/10DAS0O:AI1O013305.PNT) + 1.128*(EWS102/10DAS0A:TC1A012105.PNT) ) OR (
EWS102/10DAS0A:FT1A010401.PNT > -0.677 + 0.028*(EWS102/10DAS0O:AI1O013305.PNT) + 0.923*(EWS102/10DAS0A:FT1A010503.PNT) ))

FURNACE FRONT AND LEFT :

((( EWS102/10DAS0E:TC1E047103.PNT > 88.53 + 0.094*(EWS102/10DAS0O:AI1O013305.PNT) + 0.909*(EWS102/10DAS0D:TC1D027101.PNT) ) AND (
EWS102/10DISP:STEAMFLOWB.PNT < -69.301 + 1.084*(EWS102/10DAS0O:AI1O013305.PNT) + 254.241*(EWS102/10DAS0J:PT1J010805.PNT) )) OR (
EWS102/10DAS0A:TC1A012101.PNT > -86.282 - 0.055*(EWS102/10DAS0O:AI1O013305.PNT) + 1.128*(EWS102/10DAS0A:TC1A012105.PNT) ) OR (
EWS102/10DAS0A:FT1A010503.PNT > 8.682 - 0.019*(EWS10